# *Work in Progress* Machine Translation Project 

This project aims to translate sentences from German to English. There are in total 200,000 pairs of sentences. The current model script is named as bert_seq2seq_410.py, which contains a bert tokenizer and a vanilla 2-layer encoder-decoder architecture. The model script will continue to be updated over the next few weeks for iterations of better models, for instance transformer model.

Next step: finalize the script and add more explanation around the code blocks.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import azureml.core
from azureml.core import Workspace, Experiment, Run, Datastore

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.17


In [2]:
ws = Workspace.from_config()

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


Found the config file in: /home/nbuser/library/config.json
Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FQZZLZDSV to authenticate.
Interactive authentication successfully completed.


In [3]:
experiment_name = 'machine_translation'
exp = Experiment(workspace=ws, name=experiment_name)

In [17]:
# load config file
config_path = os.path.join(os.getcwd(), '../config.json')
with open(config_path, 'r') as f:
    config = json.load(f)

In [17]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

# choose a name for your cluster
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "gpucluster")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 4)

# This example uses CPU VM. For using GPU VM, set SKU to STANDARD_NC6/STANDARD_D2_V2
vm_size = os.environ.get("AML_COMPUTE_CLUSTER_STANDARD_NC24", "STANDARD_NC24")

if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target. just use it. ' + compute_name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    
    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
     # For a more detailed view of current AmlCompute status, use get_status()
    print(compute_target.get_status().serialize())

found compute target. just use it. gpucluster


In [25]:
ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name)

# path_on_datastore = 'machine_translation'
# ds_data = ds.path(path_on_datastore)
# print(ds_data)

# #define default datastore for current workspace
# ws.set_default_datastore('machine_translation')
ds = Datastore.register_azure_blob_container(workspace=ws, 
                                             datastore_name='machine_translation', 
                                             container_name=config["container_name"],
                                             account_name=config["account_name"], 
                                             account_key=config["account_key"],
                                             create_if_not_exists=True)

#get named datastore from current workspace
ds = Datastore.get(ws, datastore_name='machine_translation')

ds.path('./machine_translation').as_download()

AzureBlob amherstwstorageinnganzr azureml-blobstore-fe92660d-c6c1-4086-b2f7-71f9c508e6c7


$AZUREML_DATAREFERENCE_72d16a4b1fff4322aa511ab4097ca514

In [18]:
script_params = {
    '--data-folder': ds.as_mount() #ds.path('./machine_translation').as_mount() # ds.as_mount()
}

In [17]:
from azureml.train.dnn import PyTorch

pt_est = PyTorch(source_directory='./', 
                 script_params=script_params,
                 compute_target=compute_target,
                 entry_script='seq2seq_410.py', # copy_of_seq2seq.py # mt_seq2seq_attention.py # new_seq2seq.py
                 pip_packages=['torchtext','bert-embedding'],
                 use_gpu=True)

In [18]:
run = exp.submit(pt_est)

In [19]:
from azureml.core import ScriptRunConfig
from azureml.train.dnn import PyTorch

pt_est = PyTorch(source_directory='./', 
                 script_params=script_params,
                 compute_target=compute_target,
                 entry_script='copy_of_seq2seq.py', # copy_of_seq2seq.py # mt_seq2seq_attention.py # new_seq2seq.py
                 pip_packages=['torchtext','bert-embedding'],
                 use_gpu=True)

Did the run start? Queued


### To show widget of the experiment run details

In [ ]:
from azureml.widgets import RunDetails
RunDetails(run).show()

## To cancel the last run:

In [10]:
local_script_run = exp.submit(pt_est)
print("Did the run start?",local_script_run.get_status())

local_script_run.cancel()
print("Did the run cancel?",local_script_run.get_status())

Did the run cancel? Canceled


## To cancel a run based on experiment ID

In [9]:
from azureml.core import get_run
run_cpu_id = 'machine_translation_1554915123_b8da5e9d' #get from portal
run=get_run(exp, run_cpu_id)
run.cancel()